In [1]:
from collections import deque
from itertools import count
from math import lcm

## Part 1

In [2]:
test = [
"broadcaster -> a, b, c",
"%a -> b",
"%b -> c",
"%c -> inv",
"&inv -> a",]

test2 = [
    "broadcaster -> a",
"%a -> inv, con",
"&inv -> b",
"%b -> con",
"&con -> output"
        ]

In [3]:
def prepare_data(text):
    _map = {}
    conjunctions = {}
    flops = {}

    for row in text:
        module_name, dests = row.replace(" ","").split("->")
        dests = dests.split(",")

        if module_name[0] in "%":
            prefix, module_name = module_name[0], module_name[1:]
            flops[module_name] = False
        elif module_name[0] in "&":
            prefix, module_name = module_name[0], module_name[1:]
            conjunctions[module_name]= {}
            
        _map[module_name] = dests
                    
    for module_name, dests in _map.items():
        for dest in dests:
            if dest in conjunctions:
                conjunctions[dest][module_name] = False
            
    return _map, conjunctions, flops

In [4]:
def run_cycle(counts, _map):
    q = deque([["broadcaster", False]])

    while q:

        module, beam = q.popleft()
        counts[beam] = counts.get(beam,0) + 1

        dests, prefix, state = _map[module]

        if prefix is None:
            for dest in dests:
                q.append([dest, beam])

        if prefix == "%":
            if not beam:
                state = not state
                beam = state

                for dest in dests:
                    q.append([dest, beam])

        elif prefix == "&":
            _map[module] = ([dest], prefix, beam)

            beam = not all([_map[name][2] for name in conjunction])

            for dest in dests:
                q.append([dest, beam])

    return counts, _map

In [5]:
def run_cycle(_map, conjunctions, flops, counts):
    q = deque([('button', 'broadcaster', False)])

    while q:
        module_src, module_dst, beam = q.popleft()
        counts[beam] = counts.get(beam,0) + 1
        
        if module_dst in flops:
            if beam:
                continue
            next_beam = flops[module_dst] = not flops[module_dst]

        elif module_dst in conjunctions:
            conjunctions[module_dst][module_src] = beam
            next_beam = not all(conjunctions[module_dst].values())
    
        elif module_dst in _map:
            next_beam = beam

        else:
            continue

        for next_dst in _map[module_dst]:
            q.append([module_dst, next_dst, next_beam])

    return counts


In [6]:
counts = {}
_map, conjunctions, flops = prepare_data(test)

for _ in range(1000):
    counts = run_cycle(_map, conjunctions, flops, counts)
    
result = counts[False]*counts[True]
assert result == 32000000

In [7]:
counts = {}
_map, conjunctions, flops = prepare_data(test2)

for _ in range(1000):
    counts = run_cycle(_map, conjunctions, flops, counts)
    
result = counts[False]*counts[True]
assert result == 11687500

In [8]:
text = open("../advent_of_code_input/2023/day_20.txt", "r").readlines()
text = [i.split("\n")[0] for i in text]

In [9]:
counts = {}
_map, conjunctions, flops = prepare_data(text)

for _ in range(1000):
    counts = run_cycle(_map, conjunctions, flops, counts)
    
result = counts[False]*counts[True]
result

1020211150

## Part 2

shameless copy from [reddit](https://www.reddit.com/r/adventofcode/comments/18mmfxb/comment/ke5f13d/?utm_source=share&utm_medium=web2x&context=3)

In [10]:
def find_cycle(_map, conjunctions, flops):
    
    useful = set()
    
    for module_src_rx, module_dst in _map.items():
        if module_dst == ['rx']:
            assert module_src_rx in conjunctions
            break

    for module_src, module_dst in _map.items():
        if module_src_rx in module_dst:
            assert module_src in conjunctions
            useful.add(module_src)

    for iteration in count(1):
        q = deque([('button', 'broadcaster', False)])

        while q:
            module_src, module_dst, beam = q.popleft()
            
            if not beam:
                if module_dst in useful:
                    yield iteration

                    useful.discard(module_dst)
                    if not useful:
                        return

            if module_dst in flops:
                if beam:
                    continue
                next_beam = flops[module_dst] = not flops[module_dst]

            elif module_dst in conjunctions:
                conjunctions[module_dst][module_src] = beam
                next_beam = not all(conjunctions[module_dst].values())

            elif module_dst in _map:
                next_beam = beam

            else:
                continue

            for next_dst in _map[module_dst]:
                q.append([module_dst, next_dst, next_beam])

In [11]:
_map, conjunctions, flops = prepare_data(text)
cycles = list(find_cycle(_map, conjunctions, flops))
result = lcm(*cycles)
result

238815727638557